## Example tracking HARv2

In [1]:
# Import libraries
import iris
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt
import datetime
import urllib,zipfile,shutil
from netCDF4 import Dataset
import glob
%matplotlib inline
import tobac
from pathlib import Path
import xarray as xr

In [2]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [23]:
# update input parameters
from parameters import parameters_features, parameters_segmentation, dt
dxy= 10000
parameters_features['n_min_threshold'] = 10 

In [27]:
savedir = Path('out/')
file = '/media/juli/Data/projects/data/HAR v2_d10km_h_2d_prcp_2018.nc' 
Precip=iris.load_cube(file, 'total precipitation (step-wize)')

In [28]:
Precip

<iris 'Cube' of total precipitation (step-wize) / (mm h-1) (time: 8760; y-coordinate in Cartesian system: 252; x-coordinate in Cartesian system: 381)>

## Feature detection 

In [ ]:
Features= tobac.feature_detection_multithreshold(Precip,dxy,**parameters_features) 
Features.to_hdf(os.path.join(savedir,'Features_HAR.h5'),'table')

## Segmentation 

In [ ]:
Mask, Features_cells = tobac.segmentation_2D(Features, Precip, dxy, **parameters_segmentation)

iris.save([Mask], savedir/ 'Mask_Segmentation_ERA5.nc',zlib=True,complevel=4)                
Features_cells.to_hdf(savedir/'Features_cells_ERA5.h5','table')

## Download data per FTP 

In [4]:
import ftplib


In [11]:
url="ftp.klima.tu-berlin.de"
user="Anonymous"
passw="Anonymous"

In [12]:
FILENAME = 'HAR v2_d10km_h_2d_albedo_2000.nc'    

with ftplib.FTP(url, user,passw) as ftp:
    ftp.cwd('/HAR/V2/d10km/h/2d/')
    with open(FILENAME, 'wb') as f:
        ftp.retrbinary('RETR ' + FILENAME, f.write)

In [14]:
!ls *nc

'HAR v2_d10km_h_2d_albedo_2000.nc'
